In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install --upgrade fake_useragent
from bs4 import BeautifulSoup, NavigableString
import requests
import os
import io
from time import sleep
from fake_useragent import UserAgent
from PIL import Image

page and index setting

**REMEMBER TO CHANGE!**

In [3]:
id = ""
# 1st page, only main poster
url = "https://tieba.baidu.com/p/"+id+"?see_lz=1"+"&pn=1"
baseindex = 0
crawler_maximg = 30

PC browser is preferred

In [4]:
ua = UserAgent()
header = {
    "User-Agent": ua.firefox
    }
# random/chrome/firefox/opera/edge
header


{'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; rv:109.0) Gecko/20100101 Firefox/115.0'}

In [ ]:
def headerassign():
    header = {
    "User-Agent": ua.firefox
    }# random/chrome/firefox/opera/edge
    return header

In [5]:
response1 = requests.get(url, headers=header)
soup = BeautifulSoup(response1.text, "html.parser")

just for output test

In [ ]:
#soup

tags and texts

In [7]:
maxpagenum = soup.find_all("span",{"class":"red"}, limit=2)
maxpagenum = maxpagenum[1].text
maxpagenum

'6'

In [8]:
# 【图片】title【main_tieba】_百度贴吧
title_tag = soup.find("title")
title_tag

intro_tag = soup.find("div",{"class":"d_post_content j_d_post_content"})
intro = ""
for child in intro_tag.children:
    if isinstance(child, NavigableString):
        intro += child.strip()
    elif child.name == "br":
        intro += "\n"

results = soup.find_all("img", {"class": "BDE_Image"}, limit=crawler_maximg)

**REMEMBER TO ADJUST** title_full and title

In [ ]:
# print
title_full = title_tag.text[4:-5]
title_full

In [10]:
# fulltitle【main_tieba】
title = title_full[9:-11]
image_links = [result.get("src") for result in results]

folder and intro

In [11]:
# create folder
base = "/content/drive/My Drive/"+title
#!mkdir str(base)
if not os.path.exists(base):
    os.mkdir(base)

In [12]:
# save intro
introdir = base+"/intro.txt"
with open(introdir, 'w', encoding='utf-8') as f:
    f.write(intro)

In [13]:
for index, link in enumerate(image_links):
    imgresp = requests.get(link, headers=header, stream=True)
    imgpath = base +"/"+ str(baseindex+index+1) + ".jpg"
    image = Image.open(io.BytesIO(imgresp.content))
    image.save(imgpath)
    sleep(3)
    print(f"{baseindex+index+1}", end="\t")

1	2	

3	4	

5	6	

7	8	

9	10	11	

12	13	

14	

other pages if exists

In [41]:
len(image_links)

14

In [15]:
for i in range(2, int(maxpagenum)+1):
    baseindex += len(image_links)
    url = "https://tieba.baidu.com/p/"+id+"?see_lz=1"+"&pn="+str(i)
    response1 = requests.get(url, headers=header)
    soup = BeautifulSoup(response1.text, "html.parser")

    results = soup.find_all("img", {"class": "BDE_Image"}, limit=crawler_maximg)
    image_links = [result.get("src") for result in results]

    for index, link in enumerate(image_links):
        imgresp = requests.get(link, headers=header, stream=True)
        imgpath = base +"/"+ str(baseindex+index+1) + ".jpg"
        image = Image.open(io.BytesIO(imgresp.content))
        image.save(imgpath)
        sleep(3)
